## cyber~rank

In [1]:
import math
import collections
import functools
import operator

Lets say 4 agents in the network with resource token balances (2.000, 7.000, 9.000, 5.000) accor

In [2]:
agents = {
    'agent_1': 2.000,
    'agent_2': 7.000,
    'agent_3': 9.000,
    'agent_4': 5.000,
}

Named agents create cyberlinks:

In [3]:
agents_cyberlinks = {
    'agent_1': [(1, 2), (3, 4)],
    'agent_2': [(2, 3)],
    'agent_3': [(1, 3), (2, 4), (3, 4)],
    'agent_4': [(2, 1)],
}

Let's define a function to prepare the context for cyber~rank and entropy calculations:

In [4]:
def context_preparator(agents: dict, agents_cyberlinks: dict) -> list:
    cyberlinks_wihth_agent_weight = []
    for agent, cyberlinks in agents_cyberlinks.items():
        temp = (cyberlinks, agents[agent])
        cyberlinks_wihth_agent_weight.append(temp)
    grouped_cyberlinks = collections.Counter()
    for agent_cyberlinks in cyberlinks_wihth_agent_weight:
        cyberlinks = agent_cyberlinks[0]
        agent_weight = agent_cyberlinks[1]
        agent_cyberlink_weight = agent_weight / len(cyberlinks)
        agent_weighted_cyberlinks = {cyberlink: agent_cyberlink_weight for cyberlink in cyberlinks}
        grouped_cyberlinks += collections.Counter(agent_weighted_cyberlinks)
    cyberlinks = [{k:v} for k,v in grouped_cyberlinks.items()]
    return cyberlinks

And now let prepare context for the rank and entropy calculations

In [5]:
cyberlinks = context_preparator(agents, agents_cyberlinks)
cyberlinks

[{(1, 2): 1.0},
 {(3, 4): 4.0},
 {(2, 3): 7.0},
 {(1, 3): 3.0},
 {(2, 4): 3.0},
 {(2, 1): 5.0}]

Define cyber rank function:

In [6]:
def cyber_rank(cyberlinks: list, tolerance: float = 0.001, damping_factor: float = 0.8):
    cyberlinks_dict = dict(functools.reduce(operator.add, map(collections.Counter, cyberlinks)))
    objects = list(set([item for t in [list(x.keys())[0] for x in cyberlinks] for item in t]))
    size = len(objects)
    default_rank = (1.0 - damping_factor) / size
    rank = [default_rank for x in range(len(objects))]
    dangling_nodes = [obj for obj in objects if obj not in [list(cyberlink.keys())[0][1] for cyberlink in cyberlinks]]
    dangling_nodes_size = len(dangling_nodes)
    inner_product_over_size = default_rank * (dangling_nodes_size / size)
    default_rank_with_correction = (damping_factor * inner_product_over_size) + default_rank
    change = tolerance + 1

    steps = 0
    prevrank = [0] * len(objects)
    while change > tolerance:
        for obj in objects:
            obj_index = objects.index(obj)
            ksum = 0
            income_cyberlinks = [income_cyberlink for income_cyberlink in [list(x.keys())[0] for x in cyberlinks] if income_cyberlink[1] == obj]
            for cyberlink in income_cyberlinks:
                linkStake = cyberlinks_dict[cyberlink]
                outcome_cyberlinks = [outcome_cyberlink for outcome_cyberlink in [list(x.keys())[0] for x in cyberlinks] if outcome_cyberlink[0] == cyberlink[0]]
                jCidOutStake = sum([cyberlinks_dict[outcome_cyberlink] for outcome_cyberlink in outcome_cyberlinks])
                if linkStake == 0 or jCidOutStake == 0:
                    continue
                weight = linkStake / jCidOutStake
                ksum = prevrank[obj_index] * weight + ksum
            rank[obj_index] = ksum * damping_factor + default_rank_with_correction
        change = max([abs(x - y) for x, y in zip(rank, prevrank)])
        prevrank = rank.copy()
        steps += 1
    res = list(zip(objects, rank))
    res.sort(reverse=True, key=lambda x: x[1])
    res = [(cid, int(rank * 10**15)) for cid, rank in res]
    print()
    return res

In [7]:
cyber_rank(cyberlinks)

[(3, 1838759482177677),
 (4, 1246775146153395),
 (1, 68181818181818),
 (2, 62499999999999)]